In [1]:
# this notebook adapts https://github.com/BIGBALLON/CIFAR-ZOO
import time
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from easydict import EasyDict as edict

In [2]:
args = edict()
args.batch_size = 64
args.log_interval = 100
args.data_dir = './data'
args.lr = 0.003
args.momentum = 0.9
args.epochs = 50
args.lr_drop_epoch = 30
args.seed = 1
args.no_shuffle = False
print(args)

{'batch_size': 64, 'log_interval': 100, 'data_dir': './data', 'lr': 0.003, 'momentum': 0.9, 'epochs': 50, 'lr_drop_epoch': 30, 'seed': 1, 'no_shuffle': False}


In [3]:
# fix all seeds for reproducibility
seed = args.seed
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
def _init_fn(worker_id):
    np.random.seed(int(seed))

In [4]:
use_cuda = True
batch_size = args.batch_size
test_batch_size = args.batch_size

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root=args.data_dir, train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=not args.no_shuffle, **kwargs)

testset = datasets.CIFAR10(root=args.data_dir, train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size,
                                         shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
def train(args, model, device, train_loader, optimizer, epoch,
          scheduler=None):
    model.train()
    train_loss = 0
    correct = 0
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        train_loss += loss.item() * data.shape[0]  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            elapsed = time.time() - start_time
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} | ms/batch {:5.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(),
                elapsed * 1000 / args.log_interval))
            start_time = time.time()
        if scheduler is not None:
            scheduler.step()
    train_loss /= len(train_loader.dataset)
    train_acc = correct / len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    return train_loss, train_acc

In [6]:
def test(args, model, device, test_loader,
         evalmode=True):
    if evalmode:
        model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, correct / len(test_loader.dataset)

In [7]:
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        w = 16
        self.conv_1 = nn.Conv2d(3, 6 * w, 5)
        self.conv_2 = nn.Conv2d(6 * w, 16 * w, 5)
        self.fc_1 = nn.Linear(16 * 5 * 5 * w, 120 * w)
        self.fc_2 = nn.Linear(120 * w, 84 * w)
        self.fc_3 = nn.Linear(84 * w, num_classes)

    def forward(self, x):
        out = F.relu(self.conv_1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv_2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc_1(out))
        out = F.relu(self.fc_2(out))
        out = self.fc_3(out)
        return out

In [8]:
net = LeNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=args.momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, args.lr_drop_epoch)
logs = []

In [9]:
for epoch in range(1, args.epochs+1):
    epoch_start_time = time.time()
    train_loss, train_acc, = train(args, net, device, train_loader, optimizer, epoch)
    test_loss, test_acc = test(args, net, device, test_loader)
    logs.append(dict(
        epoch=epoch,
        train_loss=train_loss,
        train_acc=train_acc,
        valid_loss=test_loss,
        valid_acc=test_acc
    ))
    scheduler.step()
    print('epoch time: {:5.2f}s'.format(time.time() - epoch_start_time))

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.304694 | ms/batch  3.68
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.252081 | ms/batch 15.33
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.906888 | ms/batch 14.99
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.010757 | ms/batch 14.83
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.924497 | ms/batch 14.66
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.630958 | ms/batch 14.95
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.553795 | ms/batch 14.69
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.497387 | ms/batch 15.19

Train set: Average loss: 1.8328, Accuracy: 16714/50000 (33%)


Test set: Average loss: 1.5064, Accuracy: 4562/10000 (46%)

epoch time: 13.58s
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.507849 | ms/batch  1.32
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.473173 | ms/batch 15.00
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.571909 | ms/batch 15.07
Train Epoch: 2 [19200/50000 (38%)]	Loss: 1.371411 | ms/batch 15.08
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.416905 | ms/batch 14.7

Train Epoch: 13 [12800/50000 (26%)]	Loss: 0.035376 | ms/batch 14.63
Train Epoch: 13 [19200/50000 (38%)]	Loss: 0.184808 | ms/batch 14.66
Train Epoch: 13 [25600/50000 (51%)]	Loss: 0.181366 | ms/batch 14.83
Train Epoch: 13 [32000/50000 (64%)]	Loss: 0.117986 | ms/batch 14.59
Train Epoch: 13 [38400/50000 (77%)]	Loss: 0.156239 | ms/batch 14.62
Train Epoch: 13 [44800/50000 (90%)]	Loss: 0.126588 | ms/batch 14.62

Train set: Average loss: 0.1201, Accuracy: 48124/50000 (96%)


Test set: Average loss: 0.9660, Accuracy: 7523/10000 (75%)

epoch time: 12.91s
Train Epoch: 14 [0/50000 (0%)]	Loss: 0.035779 | ms/batch  1.05
Train Epoch: 14 [6400/50000 (13%)]	Loss: 0.041604 | ms/batch 14.62
Train Epoch: 14 [12800/50000 (26%)]	Loss: 0.086628 | ms/batch 14.85
Train Epoch: 14 [19200/50000 (38%)]	Loss: 0.018197 | ms/batch 14.73
Train Epoch: 14 [25600/50000 (51%)]	Loss: 0.086095 | ms/batch 14.82
Train Epoch: 14 [32000/50000 (64%)]	Loss: 0.198051 | ms/batch 14.69
Train Epoch: 14 [38400/50000 (77%)]	Loss: 0.082

Train Epoch: 25 [19200/50000 (38%)]	Loss: 0.000236 | ms/batch 15.03
Train Epoch: 25 [25600/50000 (51%)]	Loss: 0.000221 | ms/batch 14.96
Train Epoch: 25 [32000/50000 (64%)]	Loss: 0.000553 | ms/batch 14.84
Train Epoch: 25 [38400/50000 (77%)]	Loss: 0.000321 | ms/batch 14.78
Train Epoch: 25 [44800/50000 (90%)]	Loss: 0.000963 | ms/batch 14.81

Train set: Average loss: 0.0005, Accuracy: 50000/50000 (100%)


Test set: Average loss: 1.4757, Accuracy: 7712/10000 (77%)

epoch time: 13.18s
Train Epoch: 26 [0/50000 (0%)]	Loss: 0.000323 | ms/batch  1.19
Train Epoch: 26 [6400/50000 (13%)]	Loss: 0.000220 | ms/batch 14.74
Train Epoch: 26 [12800/50000 (26%)]	Loss: 0.000621 | ms/batch 14.69
Train Epoch: 26 [19200/50000 (38%)]	Loss: 0.000284 | ms/batch 14.54
Train Epoch: 26 [25600/50000 (51%)]	Loss: 0.000569 | ms/batch 14.62
Train Epoch: 26 [32000/50000 (64%)]	Loss: 0.000490 | ms/batch 14.61
Train Epoch: 26 [38400/50000 (77%)]	Loss: 0.000467 | ms/batch 14.62
Train Epoch: 26 [44800/50000 (90%)]	Loss: 0.00

Train Epoch: 37 [25600/50000 (51%)]	Loss: 0.000211 | ms/batch 14.66
Train Epoch: 37 [32000/50000 (64%)]	Loss: 0.000338 | ms/batch 14.71
Train Epoch: 37 [38400/50000 (77%)]	Loss: 0.000337 | ms/batch 14.70
Train Epoch: 37 [44800/50000 (90%)]	Loss: 0.000395 | ms/batch 14.51

Train set: Average loss: 0.0003, Accuracy: 50000/50000 (100%)


Test set: Average loss: 1.5368, Accuracy: 7701/10000 (77%)

epoch time: 12.90s
Train Epoch: 38 [0/50000 (0%)]	Loss: 0.000258 | ms/batch  1.24
Train Epoch: 38 [6400/50000 (13%)]	Loss: 0.000446 | ms/batch 14.61
Train Epoch: 38 [12800/50000 (26%)]	Loss: 0.000372 | ms/batch 14.53
Train Epoch: 38 [19200/50000 (38%)]	Loss: 0.000334 | ms/batch 14.56
Train Epoch: 38 [25600/50000 (51%)]	Loss: 0.000248 | ms/batch 14.79
Train Epoch: 38 [32000/50000 (64%)]	Loss: 0.000317 | ms/batch 14.70
Train Epoch: 38 [38400/50000 (77%)]	Loss: 0.000290 | ms/batch 14.63
Train Epoch: 38 [44800/50000 (90%)]	Loss: 0.000432 | ms/batch 14.72

Train set: Average loss: 0.0003, Accuracy: 50

Train Epoch: 49 [32000/50000 (64%)]	Loss: 0.000151 | ms/batch 14.75
Train Epoch: 49 [38400/50000 (77%)]	Loss: 0.000262 | ms/batch 14.68
Train Epoch: 49 [44800/50000 (90%)]	Loss: 0.000355 | ms/batch 14.69

Train set: Average loss: 0.0003, Accuracy: 50000/50000 (100%)


Test set: Average loss: 1.5471, Accuracy: 7708/10000 (77%)

epoch time: 12.94s
Train Epoch: 50 [0/50000 (0%)]	Loss: 0.000201 | ms/batch  1.32
Train Epoch: 50 [6400/50000 (13%)]	Loss: 0.000179 | ms/batch 14.58
Train Epoch: 50 [12800/50000 (26%)]	Loss: 0.000289 | ms/batch 14.64
Train Epoch: 50 [19200/50000 (38%)]	Loss: 0.000197 | ms/batch 14.80
Train Epoch: 50 [25600/50000 (51%)]	Loss: 0.000276 | ms/batch 14.70
Train Epoch: 50 [32000/50000 (64%)]	Loss: 0.000249 | ms/batch 14.58
Train Epoch: 50 [38400/50000 (77%)]	Loss: 0.000536 | ms/batch 14.76
Train Epoch: 50 [44800/50000 (90%)]	Loss: 0.000252 | ms/batch 14.64

Train set: Average loss: 0.0003, Accuracy: 50000/50000 (100%)


Test set: Average loss: 1.5482, Accuracy: 7710/10

In [10]:
logdf = pd.DataFrame(logs)

In [11]:
best_loss_epoch, best_loss = logdf.sort_values(by='valid_loss')[['epoch', 'valid_loss']].to_numpy()[0]
best_acc_epoch, best_acc = logdf.sort_values(by='valid_acc', ascending=False)[['epoch', 'valid_acc']].to_numpy()[0]
print(f'Best validation loss is {best_loss:.4f}, achieved at epoch {int(best_loss_epoch)}.\n'
      f'But best validation accuracy is {best_acc}, achieved much later at epoch {int(best_acc_epoch)}.')

Best validation loss is 0.7792, achieved at epoch 8.
But best validation accuracy is 0.7722, achieved much later at epoch 22.


In [12]:
def tight_layout(fig):
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

In [13]:
plt.figure(figsize=(8, 4))
plt.subplot(121)
sns.lineplot(data=logdf, x='epoch', y='valid_loss')
plt.axvline(best_loss_epoch, c='orange')
plt.annotate(f'validation loss\nincreases after\nepoch {int(best_loss_epoch)},\nindicating\n"overfitting"',
            xy=(best_loss_epoch, best_loss),  xycoords='data',
            xytext=(30, 1.2), textcoords='data',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='left', verticalalignment='top',
             wrap=True
            )
plt.subplot(122)
sns.lineplot(data=logdf, x='epoch', y='valid_acc')
plt.axvline(best_loss_epoch, c='orange')
plt.text(20, 0.55,
         f'...but\nvalidation\naccuracy\ncontinues\nto increase\nwell after\nepoch {int(best_loss_epoch)}!',
         horizontalalignment='left', verticalalignment='bottom')
plt.suptitle('LeNet loss and accuracy increase simultaneously on CIFAR10')
tight_layout(plt)
plt.savefig('lenet.png', dpi=200, bbox_inches='tight')